In [4]:
import pyarrow.parquet as pq
import pyarrow.csv as pv
import os
import vertica_python
from dfmultitool.sources.cdbms import vertica
from pyspark.sql import SQLContext, DataFrame, SparkSession
import subprocess
from kafka import KafkaConsumer
from kafka import KafkaProducer

In [2]:
# where ***** input your passport 
!echo 'Vreulamd1980.' | kinit petrovar

Password for petrovar@DATA.CORP: 


In [3]:
# Jobs name
JOB_NAME = "read_parquet_petrov"
# DYNAMIC_LARGE конфиг
spark = SparkSession.builder.appName(JOB_NAME)\
.config("spark.dynamicAllocation.enabled", "true")\
.config("spark.dynamicAllocation.minExecutors", "5")\
.config("spark.dynamicAllocation.maxExecutors", "20")\
.config("spark.shuffle.service.enabled", "true")\
.config("spark.executor.cores", 5)\
.config('spark.rpc.message.maxSize', '256')\
.config("spark.executor.memory", "10G")\
.enableHiveSupport().getOrCreate()

sc = spark.sparkContext

### Запись в кафку

In [ ]:
# Jobs name
JOB_NAME = "connection_kafka_2"
# DYNAMIC_LARGE конфиг
spark = SparkSession.builder.appName(JOB_NAME)\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.minExecutors", "5")\
    .config("spark.dynamicAllocation.maxExecutors", "20")\
    .config("spark.shuffle.service.enabled", "true")\
    .config("spark.executor.cores", 5)\
    .config('spark.rpc.message.maxSize', '256')\
    .config("spark.executor.memory", "10G")\
    .enableHiveSupport().getOrCreate()

sc = spark.sparkContext

file = '/data/raw/fgis_ern/broadcast/2023-05-31/broadcast.1685532297365'
logging.info(f"Путь к файлу hdfs: {file}")
df = spark.read.text(file)

TOPIC_NAME='****'
SERVER=['****']
AUTH_TYPE="PLAIN"
username='***'
password='***'

for row in df.toLocalIterator():
    producer = KafkaProducer(
            bootstrap_servers=SERVER,
            api_version=(0,11,5),
            max_block_ms=650000,  # Установить большее значение
            request_timeout_ms=300000,
            retries=1,
            max_request_size=52428800,
            sasl_mechanism=AUTH_TYPE,
            security_protocol='SASL_PLAINTEXT',
            sasl_plain_username=username,
            sasl_plain_password=password
        )

    message = ",".join(str(field) for field in row)
    logging.info(f"Отправка данных в Kafka: {message.encode('utf-8')}")
    producer.send(TOPIC_NAME, message.encode('utf-8'))

    # Закрытие Kafka producer
    producer.close()

# Закрытие сессии Spark
spark.stop()

### Просмотр топиков

In [1]:
from kafka import KafkaConsumer
import json

BOOTSTRAP_SERVERS = ['****']
consumer = KafkaConsumer(
        bootstrap_servers=BOOTSTRAP_SERVERS,
        auto_offset_reset="earliest",
        group_id="first_consumer",
    )

In [ ]:
consumer.topics()

### Чтение данных из кафки

In [7]:
i = 0
TOPIC_NAME = "*****"
BOOTSTRAP_SERVERS = ['***']


if __name__ == "__main__":
    consumer = KafkaConsumer(
        TOPIC_NAME,
        api_version=(0,11,5),
        bootstrap_servers=BOOTSTRAP_SERVERS,
        auto_offset_reset="earliest",
        group_id="first_consumer",
    )

    print("starting the consumer")
    for message in consumer:
        print(f"Message №{i+1}")
        i += 1
        print(f"MESSAGE : {message.value}", end="\n=====\n")

In [8]:
consumer.close()